# Can we predict business registration?

In [7]:
import pandas as pd 
import numpy as np
import sweetviz as sv

1) Lendo os dados 

In [11]:
df = pd.read_excel("base_empresas.xlsx")

In [10]:
df.head()

,cod_municipio_ibge,SGUF,NMMUN,qtd_ativas,qtd_ativas_Consórcio de Sociedades,qtd_ativas_Consórcio Simples,qtd_ativas_Cooperativa,qtd_ativas_EIRELI,qtd_ativas_Empresa Pública,qtd_ativas_Empresário Individual,...,valor_medio_recebido_familia,familias,valor_repassado,pib_per_capita_preco_corrente,atividade_maior_valor_ad_bruto,perc_20_59_anos,perc_idosa,perc_masculino,perc_feminino,GeoSES
0,110001,RO,Alta Floresta D'Oeste,1567,0,0,8,0,2,1102,...,18495,1676,309981,25091.18,"Administração, defesa, educação e saúde públic...",0.587227,0.129774,0.510881,0.489119,-0.468436
1,110002,RO,Ariquemes,9231,2,0,49,0,4,5942,...,17615,5245,923881,25730.20,Demais serviços,0.591797,0.098121,0.499298,0.500702,-0.271203
2,110003,RO,Cabixi,384,0,0,3,0,1,295,...,18956,296,56110,32226.25,"Administração, defesa, educação e saúde públic...",0.584567,0.143477,0.509374,0.490626,-0.401556
3,110004,RO,Cacoal,8615,1,0,26,0,4,5392,...,19376,4422,856787,29331.29,Demais serviços,0.595584,0.123658,0.489655,0.510345,-0.237966
4,110005,RO,Cerejeiras,1618,0,0,8,0,2,1158,...,17752,840,149118,37069.25,Comércio e reparação de veículos automotores e...,0.598272,0.132210,0.492292,0.507708,-0.333290
